<a href="https://colab.research.google.com/github/Huanyi77/Financial-Big-Data/blob/main/FBN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI-start-ver1

In [1]:
!pip install yfinance
!pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.8/527.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import vectorbt as vbt
data = yf.download('00915.TW', start='2020-01-01', end='2024-12-25'  ,interval='1d')

# drop the second layer in the column name
data.columns = data.columns.droplevel(1)
print(data)

/tmp/ipython-input-1307046928.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('00915.TW', start='2020-01-01', end='2024-12-25'  ,interval='1d')
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Date                                                            
2022-08-01  11.949522  11.949522  11.949522  11.949522         0
2022-08-02  11.949522  11.949522  11.949522  11.949522         0
2022-08-03  11.949522  11.949522  11.949522  11.949522         0
2022-08-04  11.949522  11.949522  11.949522  11.949522         0
2022-08-05  11.989352  11.989352  11.989352  11.989352         0
...               ...        ...        ...        ...       ...
2024-12-18  24.177275  24.196687  24.051099  24.109334  12460294
2024-12-19  23.934628  24.051099  23.789040  23.973452  19227914
2024-12-20  23.837570  23.905511  23.779336  23.895807  16373659
2024-12-23  24.099628  24.138453  23.924923  23.924923   8688869
2024-12-24  24.080215  24.186980  24.070509  24.119040   7715129

[584 rows x 5 columns]


In [4]:
data['ma12'] = data['Close'].rolling(12).mean()
data['ma26'] = data['Close'].rolling(26).mean()
data['s_out'] = (data.ma12 < data.ma26) & (data.ma12.shift(1) > data.ma26.shift(1)) # breakthrough downward
data['s_in'] = (data.ma12 > data.ma26) & (data.ma12.shift(1) < data.ma26.shift(1)) # breakthrough upward
data.dropna(inplace=True)
p = vbt.Portfolio.from_signals(data['Close'].shift(-1), data.s_in, data.s_out, freq='1d',init_cash=1000000, fees=0.004425/2, direction="longonly") #<----you have to revise this line of code!
p.positions.records

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
0,0,0,86142.316641,19,11.583069,2207.615650,51,12.100882,2306.305514,40091.666888,0.040180,0,1,0
1,1,0,84592.501683,64,12.268174,2296.122642,107,13.048768,2442.219301,61294.046494,0.059062,0,1,1
2,2,0,82284.107033,114,13.355608,2431.436338,250,17.341494,3157.081112,322386.521536,0.293358,0,1,2
3,3,0,77555.237547,260,18.317642,3143.141868,304,18.975252,3255.981791,44601.971418,0.031396,0,1,3
4,4,0,76653.298309,320,19.113758,3241.605879,365,21.865953,3708.365613,204014.880310,0.139247,0,1,4
5,5,0,70699.229125,375,23.602762,3691.992321,425,25.142460,3932.834975,101230.609438,0.060664,0,1,5
6,6,0,71769.768666,451,24.658073,3915.470599,469,24.811171,3939.780990,3132.495645,0.001770,0,1,6
7,7,0,70095.176304,472,25.291752,3922.385945,501,24.914827,3863.930389,-34206.906651,-0.019295,0,1,7


In [5]:
# Annualized return
p.annualized_return()

np.float64(0.46168665631486294)

In [6]:
p.stats()

,0
Start,2022-10-12 00:00:00
End,2024-12-24 00:00:00
Period,534 days 00:00:00
Start Value,1000000.0
End Value,1742545.285078
Total Return [%],74.254529
Benchmark Return [%],120.638152
Max Gross Exposure [%],100.0
Total Fees Paid,51456.270927
Max Drawdown [%],6.117306


In [7]:
import yfinance as yf
import pandas as pd
import vectorbt as vbt
import numpy as np

# Download data
data = yf.download('00915.TW', start='2019-01-01', end='2022-12-31', interval='1d') #<----you have to revise this line of code!
data.columns = data.columns.droplevel(1)

# Define a function to calculate portfolio performance for given parameters
def run_strategy(short_window, long_window):
    data['ma_short'] = data['Close'].rolling(short_window).mean()
    data['ma_long'] = data['Close'].rolling(long_window).mean()
    data['s_out'] = (data.ma_short < data.ma_long) & (data.ma_short.shift(1) > data.ma_long.shift(1))
    data['s_in'] = (data.ma_short > data.ma_long) & (data.ma_short.shift(1) < data.ma_long.shift(1))
    portfolio = vbt.Portfolio.from_signals(data['Close'].shift(-1), data.s_in, data.s_out, freq='1d', init_cash=1000000, fees=0.004425/2, direction='Longonly') #<----you have to revise this line of code!
    return portfolio.annualized_return()

# Parameter grid search
short_windows = range(1,10)  # Example short window values #<----you have to revise this line of code!
long_windows = range(2, 60)   # Example long window values #<----you have to revise this line of code!

best_short_window = 0
best_long_window = 0
best_annualized_return = -np.inf


for short_window in short_windows:
    for long_window in long_windows:
        if short_window < long_window: # Ensure short window is smaller than long window
            annualized_return = run_strategy(short_window, long_window)
            if annualized_return > best_annualized_return:
                best_annualized_return = annualized_return
                best_short_window = short_window
                best_long_window = long_window

print(f"Best short window: {best_short_window}")
print(f"Best long window: {best_long_window}")
print(f"Best annualized return: {best_annualized_return}")

/tmp/ipython-input-3292754552.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('00915.TW', start='2019-01-01', end='2022-12-31', interval='1d') #<----you have to revise this line of code!
[*********************100%***********************]  1 of 1 completed


Best short window: 8
Best long window: 13
Best annualized return: 0.3103408916122705


In [8]:
def run_strategy_stats(short_window, long_window,s,e):
    data = yf.download('00915.TW', start='2022-01-01', end='2024-12-25', interval='1d') #<----you have to revise this line of code!
    data.columns = data.columns.droplevel(1)
    data['ma_short'] = data['Close'].rolling(short_window).mean()
    data['ma_long'] = data['Close'].rolling(long_window).mean()
    data['s_out'] = (data.ma_short < data.ma_long) & (data.ma_short.shift(1) > data.ma_long.shift(1))
    data['s_in'] = (data.ma_short > data.ma_long) & (data.ma_short.shift(1) < data.ma_long.shift(1))
    portfolio = vbt.Portfolio.from_signals(data['Close'].shift(), data.s_in, data.s_out, freq='1d', init_cash=500000, fees=0.004425/2) #<----you have to revise this line of code!
    return portfolio.stats()


print(run_strategy_stats(8,31,'2024-1-1','2024-12-31'))

/tmp/ipython-input-1731470877.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('00915.TW', start='2022-01-01', end='2024-12-25', interval='1d') #<----you have to revise this line of code!
[*********************100%***********************]  1 of 1 completed

Start                         2022-08-01 00:00:00
End                           2024-12-24 00:00:00
Period                          584 days 00:00:00
Start Value                              500000.0
End Value                           969693.778211
Total Return [%]                        93.938756
Benchmark Return [%]                    101.67863
Max Gross Exposure [%]                      100.0
Total Fees Paid                      20722.934435
Max Drawdown [%]                         4.990627
Max Drawdown Duration           125 days 00:00:00
Total Trades                                    6
Total Closed Trades                             6
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                          40.291802
Worst Trade [%]                          0.486077
Avg Winning Trade [%]                   12.382427
Avg Losing Trade [%]                          NaN


AI-start-ver2

In [9]:
!pip install yfinance
!pip install vectorbt
!pip install groq
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt
import pandas as pd
from groq import Groq
import uuid
from datetime import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 5.4 MB/s eta 0:00:00


In [22]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import vectorbt as vbt

# 使用者輸入
asset = input("請輸入資產代碼 (如 00915.TW): ") or "00915.TW"
short_ma = int(input("請輸入短期均線天數 (如 12): ") or 12)
long_ma = int(input("請輸入長期均線天數 (如 26): ") or 26)
init_cash = float(input("請輸入初始資金 (如 1000000): ") or 1000000)
fees = float(input("請輸入交易費用率 (如 0.0022125): ") or 0.0022125)
start_date = input("請輸入回測起始日期 (如 2022-09-09): ") or "2023-01-01"
end_date = input("請輸入回測結束日期 (如 2024-12-25): ") or "2024-12-25"

# 下載數據
data = yf.download(asset, start=start_date, end=end_date, interval='1d')

# 處理多層索引
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

# 設置中文字體
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 計算均線
data[f'ma{short_ma}'] = data['Close'].rolling(short_ma).mean()
data[f'ma{long_ma}'] = data['Close'].rolling(long_ma).mean()

# 訊號產生
data['s_in'] = (data[f'ma{short_ma}'] > data[f'ma{long_ma}']) & \
               (data[f'ma{short_ma}'].shift(1) <= data[f'ma{long_ma}'].shift(1))
data['s_out'] = (data[f'ma{short_ma}'] < data[f'ma{long_ma}']) & \
                (data[f'ma{short_ma}'].shift(1) >= data[f'ma{long_ma}'].shift(1))

# 移除缺失值
data.dropna(inplace=True)

# 建立投資組合
portfolio = vbt.Portfolio.from_signals(
    close=data['Close'],
    entries=data['s_in'],
    exits=data['s_out'],
    freq='1D',
    init_cash=init_cash,
    fees=fees,
    direction="longonly"
)

# 顯示統計資料
print("\n回測統計數據：")
print(portfolio.stats().to_string())  # 使用 to_string() 可以完整顯示所有欄位


請輸入資產代碼 (如 00915.TW): 00915.TW
請輸入短期均線天數 (如 12): 12
請輸入長期均線天數 (如 26): 25
請輸入初始資金 (如 1000000): 100000
請輸入交易費用率 (如 0.0022125): 0.0022125
請輸入回測起始日期 (如 2023-01-01): 2003-01-01
請輸入回測結束日期 (如 2024-12-25): 2025-01-01


/tmp/ipython-input-3230581453.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(asset, start=start_date, end=end_date, interval='1d')
[*********************100%***********************]  1 of 1 completed



回測統計數據：
Start                                2022-09-02 00:00:00
End                                  2024-12-31 00:00:00
Period                                 565 days 00:00:00
Start Value                                     100000.0
End Value                                  184580.038085
Total Return [%]                               84.580038
Benchmark Return [%]                           101.52899
Max Gross Exposure [%]                             100.0
Total Fees Paid                              5296.629491
Max Drawdown [%]                                 4.95583
Max Drawdown Duration                   63 days 00:00:00
Total Trades                                           8
Total Closed Trades                                    8
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                        87.5
Best Trade [%]                                 29.613423
Worst Trade [%]       

In [23]:
from groq import Groq

client = Groq(api_key="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")

In [24]:
if data.empty:
    print("❌ 無法下載資料，請確認資產代碼與時間區間是否正確。")
    exit()

In [26]:
# 顯示統計資料並送出 Groq 分析
try:
    stats = portfolio.stats()
    print("\nBacktest Statistics:")
    print(stats.to_string())

    # 組出英文 Prompt
    prompt = f"""
Based on the following portfolio performance, evaluate the risk and return level (Low Risk Low Return, Low Risk Medium Return, Low Risk High Return, Medium Risk Low Return, Medium Risk Medium Return, Medium Risk High Return, High Risk Low Return, High Risk Medium Return, High Risk High Return) and provide a brief analysis:

- Asset: {asset}
- Strategy: {short_ma}-day and {long_ma}-day moving average crossover, long only
- Initial Capital: {init_cash} TWD
- Transaction Fee: {fees*100:.4f}%
- Backtest Period: {start_date} to {end_date}
- Statistics: {stats.to_dict()}

Please answer in English and explain the reasoning behind the risk evaluation.
"""

    # 發送給 Groq
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )

    print("\n📊 Risk Assessment:")
    print(response.choices[0].message.content)

except Exception as e:
    print("⚠️ An error occurred during backtest or risk analysis:", e)



Backtest Statistics:
Start                                2022-09-02 00:00:00
End                                  2024-12-31 00:00:00
Period                                 565 days 00:00:00
Start Value                                     100000.0
End Value                                  184580.038085
Total Return [%]                               84.580038
Benchmark Return [%]                           101.52899
Max Gross Exposure [%]                             100.0
Total Fees Paid                              5296.629491
Max Drawdown [%]                                 4.95583
Max Drawdown Duration                   63 days 00:00:00
Total Trades                                           8
Total Closed Trades                                    8
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                        87.5
Best Trade [%]                                 29.613423
Worst Tra